In [16]:
import numpy as np
from numpy.linalg import norm, eigh
from regularizer import GL1
from lossfunction import SimpleQudratic, LogisticLoss
from params import *
import matplotlib.pyplot as plt
import utils
from scipy.io import savemat, loadmat
import os

In [2]:
def primal(x,uk,alphak):
    return (0.5/alphak) * norm(x-uk) ** 2 + r.func(x)
def dual(y,uk,alphak):
    return -(alphak/2 * norm(y) ** 2 + uk.T@y)
def projGL1Ball(y):
    yans = y.copy()
    projected_group = []
    for i in range(r.K):
        start, end = r.starts[i], r.ends[i]
        y_Gi = y[start:end]
        norm_y_Gi = norm(y_Gi)
        if norm_y_Gi > r.Lambda_group[i]:
            yans[start:end] = (r.Lambda_group[i] / norm_y_Gi) * y_Gi
            projected_group.append(i)
    return yans, projected_group
def prox_gl1(uk):
    prox = np.zeros_like(xk)
    gradstep = uk
    for i in range(len(r.starts)):
        start, end = r.starts[i], r.ends[i]
        gradstep_Gi = gradstep[start:end]
        prox[start:end] = max(0, 1 - (r.Lambda_group[i] * alphak/np.sqrt(np.sum(gradstep_Gi*gradstep_Gi)))) * gradstep_Gi
    return prox            

In [3]:
# setup problems
loss = 'logit'
datasetName = 'diabetes'
frac = 0.3; lam_shrink = 0.8
fileType = fileTypeDict[datasetName]
print("Working on: {}...".format(datasetName))
X, y = utils.set_up_xy(datasetName, fileType, dbDir='/Users/ym/Documents/GroupFaRSA/db/')
if loss == 'logit':
    f = LogisticLoss(X, y, datasetName)

p = X.shape[1]
num_of_groups = max(int(p * frac), 2)
group = utils.gen_group(p, num_of_groups)
lammax_path = f'/Users/ym/Documents/GroupFaRSA/db/lammax-{datasetName}-{frac}.mat'
Lip_path = f'/Users/ym/Documents/GroupFaRSA/db/Lip-{datasetName}.mat'
if os.path.exists(lammax_path):
    lammax = loadmat(lammax_path)["lammax"][0][0]
    print(f"loading lammax from: {lammax_path}")
else:
    lammax = utils.lam_max(X, y, group, loss)
    savemat(lammax_path, {"lammax": lammax})
    print(f"save lammax to: {lammax_path}")
if os.path.exists(Lip_path):
    L = loadmat(Lip_path)["L"][0][0]
    print(f"loading Lipschitz constant from: {Lip_path}")
else:
    L = utils.estimate_lipschitz(X, loss)
    savemat(Lip_path, {"L": L})
    print(f"save Lipschitz constant to: {Lip_path}")
Lambda = lammax * lam_shrink
r = GL1(Lambda=Lambda, group=group)

Working on: diabetes...
loading lammax from: /Users/ym/Documents/GroupFaRSA/db/lammax-diabetes-0.3.mat
loading Lipschitz constant from: /Users/ym/Documents/GroupFaRSA/db/Lip-diabetes.mat


In [4]:
# initialization
k = 0
# constant stepsize rule
alphak = 1/L * 0.8
xk = np.zeros((p,1))
y_last_round = xk.copy()
while True:
    fxk = f.func(xk)
    rxk = r.func(xk)
    gradfxk = f.grad(xk)
    uk = xk - alphak * gradfxk
    prox_actual = prox_gl1(uk)
    # constant stepsize rule
    stepsize = 1 / alphak
    y_current = y_last_round
    inner_iters = 0
    while True:
        inner_iters += 1
        ytrial, projected_group = projGL1Ball(y_current - stepsize * (alphak * y_current + uk))
        xtrial = alphak * ytrial + uk
        xtrial_proj = xtrial.copy()
        for i in range(r.K):
            if i not in projected_group:
                start, end = r.starts[i], r.ends[i]
                xtrial_proj[start:end] = 0.0
        # check for termination
        dualVal = dual(ytrial, uk, alphak)
        primalValProj = primal(xtrial_proj, uk, alphak)
        gapProj = (primalValProj - dualVal)[0][0]
        if gapProj < 1/(k+1)**3:
            xtrial = xtrial_proj
            gap = gapProj
            break
        primalVal = primal(xtrial, uk, alphak)
        gap = (primalVal - dualVal)[0][0]
        if gap < 1/(k+1)**3:
            break
        if inner_iters > 3:
            break
        y_current = ytrial
        stepsize *= 0.8
    y_last_round = ytrial
    optim_approx = norm(xtrial - xk)
    optim_actual = norm(prox_actual - xk)
    print(f"k:{k:3d} | F:{fxk+rxk:3.3e}| subits:{inner_iters:4d} | stepsize:{stepsize:3.3e} | gap:{gap:+3.3e} | tarGap:{1/(k+1)**3:+3.3e} | OptimApprox: {optim_approx:3.3e} | Optim: {optim_actual:3.3e} ")
    if optim_actual < 1e-5:
        break
    k += 1
    xk = xtrial

k:  0 | F:6.931e-01| subits:   1 | stepsize:7.159e-01 | gap:+0.000e+00 | tarGap:+1.000e+00 | OptimApprox: 6.508e-02 | Optim: 6.508e-02 
k:  1 | F:6.909e-01| subits:   1 | stepsize:7.159e-01 | gap:+1.388e-17 | tarGap:+1.250e-01 | OptimApprox: 3.057e-02 | Optim: 3.057e-02 
k:  2 | F:6.904e-01| subits:   1 | stepsize:7.159e-01 | gap:+0.000e+00 | tarGap:+3.704e-02 | OptimApprox: 1.457e-02 | Optim: 1.457e-02 
k:  3 | F:6.903e-01| subits:   1 | stepsize:7.159e-01 | gap:+0.000e+00 | tarGap:+1.562e-02 | OptimApprox: 7.029e-03 | Optim: 7.029e-03 
k:  4 | F:6.903e-01| subits:   1 | stepsize:7.159e-01 | gap:+6.939e-18 | tarGap:+8.000e-03 | OptimApprox: 3.423e-03 | Optim: 3.423e-03 
k:  5 | F:6.903e-01| subits:   1 | stepsize:7.159e-01 | gap:+6.939e-18 | tarGap:+4.630e-03 | OptimApprox: 1.677e-03 | Optim: 1.677e-03 
k:  6 | F:6.903e-01| subits:   1 | stepsize:7.159e-01 | gap:+0.000e+00 | tarGap:+2.915e-03 | OptimApprox: 8.246e-04 | Optim: 8.246e-04 
k:  7 | F:6.903e-01| subits:   1 | stepsize:7.15

In [5]:
xk.T

array([[ 0.        ,  0.        ,  0.        ,  0.        , -0.07203054,
        -0.02478641, -0.06501051, -0.0728183 ]])

In [6]:
print("#nz:", utils.get_group_structure(xtrial, r.K, r.starts, r.ends))

#nz: 1


In [10]:
# setup problems
Q = np.diag([1,1,2,2,]); b = np.array([1/2, 1/2, -33/5, -44/5]).reshape(-1,1); c=0
f = SimpleQudratic(Q,b,c)
Lambda_group = np.array([1.0,1.0]); group = np.array([1,1,2,2])
r = GL1(group, Lambda_group=Lambda_group)

# initialization
k = 0
# constant stepsize rule
alphak = 1/np.max(eigh(Q)[0]) * 0.8
xk = np.ones((Q.shape[0],1)) + 0.02
y_last_round = xk.copy()
xseq = []
yseq = []
xseq.append(xk)
yseq.append(y_last_round)
while True:
    gradfxk = f.grad(xk)
    uk = xk - alphak * gradfxk
    prox_actual = prox_gl1(uk)
    # projected gradient solve the dual proximal gradient problem
    # http://www.princeton.edu/~yc5/ele522_optimization/lectures/grad_descent_constrained.pdf
    # constant stepsize rule
    stepsize = 1 / alphak * 0.1
    y_current = y_last_round
    inner_iters = 0
    while True:
        inner_iters += 1
        ytrial, projected_group = projGL1Ball(y_current - stepsize * (alphak * y_current + uk))
        xtrial = alphak * ytrial + uk
        xtrial_proj = xtrial.copy()
        for i in range(r.K):
            if i not in projected_group:
                start, end = r.starts[i], r.ends[i]
                xtrial_proj[start:end] = 0.0
        # check for termination
        dualVal = dual(ytrial, uk, alphak)
        primalValProj = primal(xtrial_proj, uk, alphak)
        gapProj = (primalValProj - dualVal)[0][0]
        if gapProj < 1/(k+1)**3:
            print("projection accepted!")
            xtrial = xtrial_proj
            gap = gapProj
            break
        primalVal = primal(xtrial, uk, alphak)
        gap = (primalVal - dualVal)[0][0]
        if gap < 1/(k+1)**3:
            print("projection not accepted!")
            break
        if inner_iters > 1000:
            break
        y_current = ytrial
        stepsize *= 0.8
    y_last_round = ytrial
    optim_approx = norm(xtrial - xk)
    optim_actual = norm(prox_actual - xk)
    print(f"k:{k:3d} | subits:{inner_iters:4d} | projGrp:{projected_group} | gap:{gap:+3.3e} | tarGap:{1/(k+1)**3:+3.3e} | OptimApprox: {optim_approx:3.3e} | Optim: {optim_actual:3.3e} ")
    if optim_actual < 1e-5:
        break
    k += 1
    xk = xtrial
    xseq.append(xk)
    yseq.append(y_last_round)


projection accepted!
k:  0 | subits:   3 | projGrp:[1] | gap:+9.191e-01 | tarGap:+1.000e+00 | OptimApprox: 3.205e+00 | Optim: 3.127e+00 
projection accepted!
k:  1 | subits:   1 | projGrp:[1] | gap:+4.274e-03 | tarGap:+1.250e-01 | OptimApprox: 5.725e-01 | Optim: 5.736e-01 
projection accepted!
k:  2 | subits:   1 | projGrp:[1] | gap:+8.845e-04 | tarGap:+3.704e-02 | OptimApprox: 1.153e-01 | Optim: 1.163e-01 
projection accepted!
k:  3 | subits:   1 | projGrp:[1] | gap:+3.323e-04 | tarGap:+1.562e-02 | OptimApprox: 2.375e-02 | Optim: 2.445e-02 
projection accepted!
k:  4 | subits:   1 | projGrp:[1] | gap:+2.130e-04 | tarGap:+8.000e-03 | OptimApprox: 5.233e-03 | Optim: 5.684e-03 
projection accepted!
k:  5 | subits:   1 | projGrp:[1] | gap:+1.644e-04 | tarGap:+4.630e-03 | OptimApprox: 1.342e-03 | Optim: 1.587e-03 
projection accepted!
k:  6 | subits:   1 | projGrp:[1] | gap:+1.320e-04 | tarGap:+2.915e-03 | OptimApprox: 4.182e-04 | Optim: 5.283e-04 
projection accepted!
k:  7 | subits:   1 

In [ ]:
# initialization
k = 0
# constant stepsize rule
alphak = 1/np.max(eigh(Q)[0]) * 0.8
xk = np.ones((Q.shape[0],1)) + 0.02
y_last_round = xk.copy()
xseq = []
yseq = []
xseq.append(xk)
yseq.append(y_last_round)
while True:
    gradfxk = f.grad(xk)
    uk = xk - alphak * gradfxk
    prox_actual = prox_gl1(uk)
    # projected gradient solve the dual proximal gradient problem
    # http://www.princeton.edu/~yc5/ele522_optimization/lectures/grad_descent_constrained.pdf
    # constant stepsize rule
    stepsize = 1/alphak * 0.1
    y_current = y_last_round
    inner_iters = 0
    while True:
        inner_iters += 1
        ytrial, projected_group = projGL1Ball(y_current - stepsize * (alphak * y_current + uk))
        xtrial = alphak * ytrial + uk
        # check for termination
        primalVal = primal(xtrial, uk, alphak)
        dualVal = dual(ytrial, uk, alphak)
        gap = (primalVal - dualVal)[0][0]
        if gap < 1/(k+1)**3:
            break
        if inner_iters > 1000:
            break
        y_current = ytrial
    y_last_round = ytrial
    optim_approx = norm(xtrial - xk)
    optim_actual = norm(prox_actual - xk)
    print(f"k:{k:3d} | subits:{inner_iters:4d} | projGrp:{projected_group} |gap:{gap:+3.3e} | tarGap:{1/(k+1)**3:+3.3e} | OptimApprox: {optim_approx:3.3e} | Optim: {optim_actual:3.3e} ")
    if optim_actual < 1e-5:
        break
    k += 1
    xk = xtrial
    xseq.append(xk)
    yseq.append(y_last_round)

# Overlapping

In [16]:
from scipy.sparse import csc_matrix
import spams
# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
from numpy.linalg import norm, eigh
param = {'numThreads' : -1,'verbose' : True,
         'pos' : False, 'intercept' : False, 'lambda1' : 1}
m = 10
n = 1
np.random.seed(1234)
U = np.asfortranarray(np.random.normal(size = (m,n)),dtype=np.float64)
# Example 1 of tree structure
# tree structured groups:
# g1= {0 1 2 3 4 5 6 7 8 9}
# g2= {2 3 4}
# g3= {5 6 7 8 9}
own_variables =  np.array([0,2,5],dtype=np.int32) # pointer to the first variable of each group
N_own_variables =  np.array([2,3,5],dtype=np.int32) # number of "root" variables in each group
# (variables that are in a group, but not in its descendants).
# for instance root(g1)={0,1}, root(g2)={2 3 4}, root(g3)={5 6 7 8 9}
eta_g = np.array([1,30,1],dtype=np.float64) # weights for each group, they should be non-zero to use fenchel duality
groups = np.asfortranarray([[0,0,0],
                            [1,0,0],
                            [1,0,0]],dtype = np.bool)
# first group should always be the root of the tree
# non-zero entriees mean inclusion relation ship, here g2 is a children of g1,
# g3 is a children of g1
groups = csc_matrix(groups,dtype=np.bool)
tree = {'eta_g': eta_g,'groups' : groups,'own_variables' : own_variables,
        'N_own_variables' : N_own_variables}

param['regul'] = 'tree-l2'
alpha = spams.proximalTree(U,tree,False,**param)
print(alpha.T)

[[ 0.24474511 -0.61829389  0.          0.          0.          0.28600374
   0.27711426 -0.20520255  0.0050602  -0.72299715]]


In [2]:
def groups_to_matrix(groups):
    lifted_dimension = sum([len(g) for g in groups])
    groups_flattern = []
    for g in groups:
        groups_flattern += g
    p = max(groups_flattern) + 1
    A = np.zeros((p, lifted_dimension))
    for (colidx, rowidx) in enumerate(groups_flattern):
        A[rowidx, colidx] = 1.0
    #return lifted_dimension, groups_flattern, A
    return A
def relabel(groups):
    group_size = [len(g) for g in groups]
    starts, ends = [] * len(groups) , [] * len(groups)
    start = 0
    for i in group_size:
        starts.append(start)
        end = start + i 
        ends.append(end)
        start = end 
    return starts, ends
def projGL1Ball(y, starts, ends, weights, K):
    yans = y.copy()
    projected_group = []
    for i in range(K):
        start, end = starts[i], ends[i]
        y_Gi = y[start:end]
        norm_y_Gi = norm(y_Gi)
        if norm_y_Gi > weights[i]:
            yans[start:end] = (weights[i] / norm_y_Gi) * y_Gi
            projected_group.append(i)
    return yans, projected_group
def primal(x,uk,alphak):
    return (0.5/alphak) * norm(x-uk) ** 2 + r.func(x)
def dual(y,uk,alphak):
    return -(alphak/2 * norm(y) ** 2 + uk.T@y)
# groups = [[0,1,2], [1,2,3], [2,4]]
# A = groups_to_matrix(groups)
# relabel(groups)

In [3]:
class natOG:
    def __init__(self, groups, penalty=1.0, weights=None):
        assert len(groups) == len(weights), "should be of the same lengthg"
        self.penalty = penalty
        self.groups = groups
        self.K = len(groups)
        if weights is None:
            weights = np.array([np.sqrt(len(g)) for g in groups])
        self.weights = self.penalty * weights
    def func(self, x):
        ans = 0.0
        for i,g in enumerate(self.groups):
            xg = x[g]
            ans += norm(xg) * self.weights[i]
        return ans

In [4]:
g0= [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
g1= [2, 3, 4]
g2= [5, 6, 7, 8, 9]
groups = [g0, g1, g2]
groups_dict = {}
for i in range(len(groups)):
    groups_dict[i] = groups[i]
A = groups_to_matrix(groups)
starts, ends = relabel(groups)
weights = np.array([1.0,30.0,1.0])
r = natOG(groups, penalty=1.0, weights=weights)

In [10]:
alphak = 1.0
stepsize = 1 / alphak 
# stepsize = 1 / ( alphak * max(eigh(A.T@A)[0]))
lifted_dim = A.shape[1]
y_lifted = np.zeros((lifted_dim,  1))
inner_iters = 0
np.random.seed(1234)
uk = np.random.normal(size = (m,n))
while True:
    inner_iters += 1
    y_lifted_trial, projected_group = projGL1Ball(y_lifted - stepsize * (alphak * A.T@A@y_lifted + A.T@uk), starts, ends, r.weights, len(groups))
    xtrial = alphak * (A@y_lifted_trial) + uk
    xtrial_proj = xtrial.copy()
    for g in range(r.K):
        if g not in projected_group:
            xtrial_proj[groups_dict[g]] = 0.0
    # check for termination
    primalVal = primal(xtrial, uk, alphak)
    dualVal = dual(A@y_lifted_trial, uk, alphak)
    gap = (primalVal - dualVal)[0][0]
    if gap < 1e-10:
        break
    if inner_iters > 1000:
        break
    y_lifted = y_lifted_trial
print(inner_iters,gap)


72 9.896261587982735e-11


In [11]:
print(xtrial_proj.T)
print("diff:")
print((xtrial_proj - alpha).T)

[[ 0.24474511 -0.61829389  0.          0.          0.          0.28600374
   0.27711426 -0.20520255  0.0050602  -0.72299715]]
diff:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -5.55111512e-17 -5.55111512e-17  2.77555756e-17
  -8.67361738e-19 -1.11022302e-16]]


In [28]:
# g0, g1, g2 = [0,1,3], [2,4,5], [6,7]
# g0, g1, g2 = [0,1,3,6], [0,1,2,3,6,7], [0,1,3,4,5,6,7,8]
# g0, g1, g2 = [0, 1, 2, 3, 4], [3, 4, 5, 6, 7, 8], [6, 7, 8, 9]
groups = [g0, g1, g2]
A = groups_to_matrix(groups)
lam, _ = eigh(A.T@A)
lam

array([1., 1., 1., 1., 1., 1., 1., 1.])

In [55]:
xk = np.array([-1.1, -2.2, -3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9, 11.4]).reshape(-1,1)
gradfxk = 0.01*np.array([11.1, 2.2, 33.3, -44.4, -5.5, 36.6, 77.7, 8.8, 9.9, 11.4]).reshape(-1,1)
alphak = 0.2;
uk = xk - alphak * gradfxk
g0= [0, 1, 2, 3, 4]
g1= [3, 4, 5, 6, 7, 8]
g2= [6, 7, 8, 9]
groups = [g0, g1, g2]
groups_dict = {}
for i in range(len(groups)):
    groups_dict[i] = groups[i]
A = groups_to_matrix(groups)
starts, ends = relabel(groups)
weights = np.array([np.sqrt(1000),np.sqrt(2),np.sqrt(10)]) * 2.0
r = natOG(groups, penalty=1.0, weights=weights)

In [56]:
starts, ends

([0, 5, 11], [5, 11, 15])

In [57]:
stepsize = 1 / alphak
lifted_dim = A.shape[1]
y_lifted = np.zeros((lifted_dim,  1))
inner_iters = 0
while True:
    inner_iters += 1
    y_lifted_trial, projected_group = projGL1Ball(y_lifted - stepsize * (alphak * A.T@A@y_lifted + A.T@uk), starts, ends, r.weights, len(groups))
    xtrial = alphak * (A@y_lifted_trial) + uk
    xtrial_proj = xtrial.copy()
    for g in range(r.K):
        if g not in projected_group:
            xtrial_proj[groups_dict[g]] = 0.0
    # check for termination
    primalVal = primal(xtrial, uk, alphak)
    dualVal = dual(A@y_lifted_trial, uk, alphak)
    gap = (primalVal - dualVal)[0][0]
    if gap < 1e-10:
        break
    if inner_iters > 1000:
        break
    y_lifted = y_lifted_trial
print(inner_iters,gap)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
17 6.440359356929548e-11


In [58]:
xtrial_proj

array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 6.29082982],
       [ 6.7938454 ],
       [ 7.90847332],
       [ 8.89703248],
       [10.60321887]])

In [48]:
uk

array([[-1.1222],
       [-2.2044],
       [-3.3666],
       [ 4.4888],
       [ 5.511 ],
       [ 6.5268],
       [ 7.5446],
       [ 8.7824],
       [ 9.8802],
       [11.3772]])

array([[inf],
       [inf],
       [inf]])

In [46]:
import yaml
with open('./config.yaml', "r") as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

In [49]:
inexact_strategy_name = config['mainsolver']['inexact_strategy']
config['inexactpg'][inexact_strategy_name]

{'delta': 3, 'c': 1.0}

In [50]:
A = np.diag([1,2]); B = np.array([[1,2],[3,4]])

In [51]:
A, B

(array([[1, 0],
        [0, 2]]),
 array([[1, 2],
        [3, 4]]))

In [52]:
A@B, B@A

(array([[1, 2],
        [6, 8]]),
 array([[1, 4],
        [3, 8]]))